## Discover

1. covid presents us with new challenges
2. we need to be able to adapt to the ever changing conditions that we are presented with
3. to do so, we need to be able to know when is the ideal time to start our ideas and seize the opportunities in this uncertain time (model uncertain timings hehhehe)

## Define

1. Singapore, where we are the most familiar with.
2. When are we able to open up our economy given our government's stance, number of cases, vaccination rates
3. When are we able to flatten the curve

## Develop

1. To do so, we can come up with a multiple linear regression model to calculate what would the stringency index be to allow ease of business and daily activities
2. This will allow opportunity seekers to have a good idea of when Singapore is able to flatten the curve and resume normal life

## Deliver

1. Provide a model for users to key in various parameters and calculate the current expected stringency index

## Problem Statement

### Given a rise in covid-19 cases, the Singaporean Government would always take the necessary precautions to prevent an unpreceeded rise in COVID-19 transmission. This increases the Stringency Index significantly as workplaces, schools closes. In such a case, what would the stringency index of Singapore be?

#### Start with importing Data that outlines Singapore's Covid-19 response

### Data

#### Link to data source: https://github.com/owid/covid-19-data/tree/master/public/data

#### Why is this data used?
1. Sources are clearly documented, for example, all data under Singapore comes directly from Ministry of Health, Singapore, which ensures that data is accurate and not tweaked or false
2. Contains very little missing values and it updated daily
    - Ensures that the data given is current with little errors
    - Missing data is filled using growth trendline analysis using excel
   - Ensures a growth trend in data such as vaccinations rates using values before and after, which can only be increasing
3. Provides a large variety of parameters
    - Ensures that we have a diverse range of parameters to accurately train our model
    - Provides smoothed values to ensure no abnormal values
      - 0 vaccinations prior to discovery of vaccinations and deflivery to Singapore on 31 Dec 2020

#### Model

Features considered:
1. new_cases_smoothed_per_million
    - The number of new cases directly corresponds to the community transmission of COVID-19, giving a clear indication of the prevalence of virus in the community
    - Higher number of new cases signals high transmission rates in the community, leading to higher stringency index to keep new cases low
2. total_cases_per_millon
    - Total numnber of infected individuals is an indication of how many people in the community have natural immunity to COVID-19
    - Higher total cases will lead to herd immnunity, leading to lower stringency index
3. new_deaths_smoothed_per_million
    - Indicates the number of new deaths, an important indicator that is in line with Singapore's stance to be COVID resilient with keeping daily death rates low
    - Higher death rates will signal that the virus is becoming more deadly, leading to a higher stringency index to keep people safe
4. new_tests_smoothed_per_thousand
    - Indicates the number of test done each day, a higher number of testing done will help to ensure that positive cases are picked up earlier and therefore isolated and reduce the spread of the virus
    - Higher number of new tests will lower transmissibility leading to stringency index lowering
5. positive_rate
    - Indicates the percentage of test done that are positive, will give an indication of how many cases are there in the community that is transmitted without being picked up
    - Higher postive rates with higher reservoir of undetected cases will lead to stringency index increasing
6. people_fully_vaccinated_per_hundred
    - Indicates how many people are fully vaccinated and therefore lowering the chances of death as well as having a shorter infectious period, leading to slower transmission rates
    - Higher vaccination rates gives lower death rate and lower transmissibility will likely lead to lower stringency index

- Perform various test to ensure we find the optimal alpha value (learning rate), beta value (starting weight) and number of iterations (times gradient descent is performed)


#### Model Evaluation

1. Mean Squared Error
2. R<sup>2</sup> value
      


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# data here is filled with growth trendline analysis using excel and 0's where places are obvious
df = pd.read_csv("Data/Singapore_Covid_Data_csv.csv")
display(df)

,location,date,new_cases_smoothed_per_million,total_cases_per_million,new_deaths_smoothed_per_million,reproduction_rate,new_tests_smoothed_per_thousand,positive_rate,people_fully_vaccinated_per_hundred,stringency_index
0,Singapore,1/3/2020,0.445,0.183,0.000,0.56,0.000,0.000,0.000,30.56
1,Singapore,2/3/2020,0.498,0.550,0.000,0.54,0.000,0.000,0.000,30.56
2,Singapore,3/3/2020,0.498,0.550,0.000,0.53,0.000,0.000,0.000,30.56
3,Singapore,4/3/2020,0.445,0.733,0.000,0.57,0.000,0.000,0.000,30.56
4,Singapore,5/3/2020,0.629,0.917,0.000,0.73,0.000,0.000,0.000,30.56
...,...,...,...,...,...,...,...,...,...,...
609,Singapore,31/10/2021,673.998,15193.634,0.288,0.97,4.130,0.163,86.405,44.44
610,Singapore,1/11/2021,655.556,15496.186,0.367,0.94,4.130,0.159,86.445,44.44
611,Singapore,2/11/2021,661.293,15760.782,0.419,0.92,3.976,0.166,86.485,44.44
612,Singapore,3/11/2021,617.050,16116.327,0.472,0.90,3.822,0.161,86.525,44.44


## Start of manual ML code

In [76]:
def normalize_z(df):
    return ((df - df.mean(axis=0))/df.std(axis=0))

def get_features_targets(df, feature_names, target_names):
    # get df of selected features
    df_feature = df[feature_names]
    # get df of selected targets
    df_target = df[target_names]
    return df_feature, df_target

def prepare_feature(df_feature):
    # number of columns in the dataframe
    cols = len(df_feature.columns)
    # convert df to numpy
    feature = df_feature.to_numpy().reshape(-1,cols)
    array = np.concatenate((np.ones((feature.shape[0],1)), feature), axis = 1)
    return array

def prepare_target(df_target):
    cols = len(df_target.columns)
    target = df_target.to_numpy().reshape(-1,cols)
    return target

def predict(df_feature, beta):
    df_feature = normalize_z(df_feature)
    preped_feature = prepare_feature(df_feature)
    return predict_norm(preped_feature, beta)

def predict_norm(X, beta):
    return np.matmul(X,beta)

def split_data(df_feature, df_target, random_state=100, test_size=0.3):
    indexes = df_feature.index
    if random_state != None:
        np.random.seed(random_state)
    k = int(test_size * len(indexes))
    test_index = np.random.choice(indexes, k, replace=False)
    indexes = set(indexes)
    test_index = set(test_index)
    train_index = indexes - test_index
    # the above indexes just helps you to get random indexes within the entire data
    df_feature_train = df_feature.loc[train_index, :]
    df_feature_test = df_feature.loc[test_index, :]
    df_target_train = df_target.loc[train_index, :]
    df_target_test = df_target.loc[test_index, :]
    
    return df_feature_train, df_feature_test, df_target_train, df_target_test
  
def r2_score(y, ypred):
    ss_res = np.sum((y-ypred)**2)
    y_mean = np.mean(y)
    ss_tot = np.sum((y-y_mean)**2)
    r_2 = (1-(ss_res/ss_tot))
    return r_2

def mean_squared_error(target, pred):
    num_data = target.shape[0]
    return (1/num_data)*(np.sum((target-pred)**2))

def compute_cost(X, y, beta): #beta is weighted values, in this case it is just choosen from random values
    J = 0
    number_of_samples = X.shape[0]
    error = np.matmul(X, beta) - y
    error_sq = np.matmul(error.T, error)
    J = (1)/(2*number_of_samples) * error_sq 
    J = J[0][0]
    return J

def gradient_descent(X, y, beta, alpha, num_iters):
    number_of_samples = X.shape[0]
    J_storage = []
    for i in range(num_iters):
        derivative_error = (1/(number_of_samples)) * np.matmul(X.T, (np.matmul(X, beta) - y))
        beta = beta - alpha *  derivative_error
        J_storage.append(compute_cost(X, y, beta))
    return beta, J_storage

In [77]:
# Extract the features and the targets
df_features, df_target = get_features_targets(df.loc[46:,:],["new_cases_smoothed_per_million","total_cases_per_million","new_deaths_smoothed_per_million", "reproduction_rate", "new_tests_smoothed_per_thousand","people_fully_vaccinated_per_hundred","positive_rate"],["stringency_index"])

# Normalize using z normalization
df_features = normalize_z(df_features)

In [78]:
# Instantiate default values for alpha, beta and number of iterations
alpha = 0.01
iterations = 1500
beta = np.ones((8,1))


In [79]:
# Extract the features and the target
df_features_original_train, df_target = get_features_targets(df.loc[46:,:],["new_cases_smoothed_per_million","total_cases_per_million","new_deaths_smoothed_per_million", "reproduction_rate", "new_tests_smoothed_per_thousand","people_fully_vaccinated_per_hundred","positive_rate"],["stringency_index"])

# Split the data set into training and test
df_features_train, df_features_test, df_target_train, df_target_test = split_data(df_features_original_train,df_target,100,0.3)

# Normalize the features using z normalization
df_features_train_z = normalize_z(df_features_train)

# Change the features and the target to numpy array using the prepare functions
X = prepare_feature(df_features_train_z)
target = prepare_target(df_target_train)

# Call the gradient_descent function
beta, J_storage = gradient_descent(X, target, beta, alpha, iterations)

# call the predict() method
pred = predict(df_features_test,beta)

In [80]:
# change target test set to a numpy array
target = prepare_target(df_target_test)

# Calculate r2 score by calling a function
r2 = r2_score(target, pred)
print("r^2 value:" , r2)

r^2 value: 0.85085930974259


In [81]:
# Calculate the mse
mse = mean_squared_error(target, pred)
print("mean squared error:", mse)

mean squared error: 11.371266401850097


## Ways to improve model

1. Use most suitable beta-value 
2. Optimal Number of iterations and alpha-value

### Calculate the r-squared value over a range of beta, using different constants such as 0's, 1's, 10's

- Guess: Having more suitable values for beta will be able to generate better beta values with minimal number of iterations of gradient descent

In [ ]:
# 